# Seyed Mohammad Amin Atyabi - 830402014 - HW 2

در این مسئله، حالت ها، عمل ها و آزمون هدف بدین صورت تعیین می گردد:
- حالت ها: هر چینش از پازل را یک حالت می نامیم که حاوی اعداد 1 تا 8 بعلاوه یک محل خالی در یک آرایه دو بعدی 3 در 3 است
- عمل ها: جابجایی هر عدد مجاور به خانه خالی به آن خانه یک عمل نامیده میشود
- آزمون هدف: محاسبه فاصله منهتن یک حالت به حالت نهایی آزمون هدف است که در صورت صفر شدن به معنی رسیدن به هدف است

تابع `next_node` در کلاس `Node` پیاده سازی شده است و به ازای هر حالت، حالات فرزند را تولید می کند.

حالت فرزند از جابجا کردن خانه های مجاور به خانه خانه خالی با آن تولید میشود.



تابع `manhattan_distance` در کلاس `Node` پیاده سازی شده است و به ازای هر حالت، حالات فرزند را تولید می کند.

این تابع با نام `a_star` پیاده سازی شده است.

تا زمانی که لیست گره های ملاقات نشده پر است هر بار حالتی با کمترین مقدار $f + g$ را انتخاب می کنیم. این گره در آخر لیست است.

سپس آزمون هدف را انجام داده و در صورت موفقیت مسیر را بر می گردانیم.

در صورتی که حالت فعلی قبلا در لیست توسعه داده شده ها بود ادامه خواهیم داد به حالت بعدی.

سپس به ازای آن گره، گره های فرزند را تولید میکنیم. اگر گره فرزند تولید شده در لیست توسعه داده شده ها بود ولی با مقدار $g$ بیشتر، آن را از لیست توسعه داده شده های حذف میکنیم. گره های فرزند را به لیست گره های ملاقات نشده اضافه میکنیم.

در آخر گره فعلی را به لیست گره های توسعه داده شده اضافه میکنیم و پس از آن لیست گره های ملاقات نشده را مرتب کرده و مراحل را از اول انجام میدهیم.

In [1]:
import copy

EMPTY = 0
MANHATTAN_HEURISTIC = 1
MANHATTAN_LINEAR_CONFLICT_HEURISTIC = 2


def print_state(state):
    for i in state:
        print(str(i.state).replace('[', '').replace(']', '').replace(',', ' |').replace('0', ' '))


def print_path(path):
    for i in path:
        print(
            str(i.state).replace('0', ' ').replace('], [', '\n').replace('[[', '').replace(']]', '').replace(',', ' |'))
        if i != path[-1]:
            print('    ↓')


class Node:
    def __init__(self, state, g, h, parent):
        self.state = state
        self.g = g
        self.h = h
        self.value = g + h
        self.parent = parent

    def next_node(self, goal, heuristic):
        new_states = []
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                new_state = None
                if j + 1 < len(self.state[i]) and self.state[i][j + 1] == EMPTY:
                    new_state = copy.deepcopy(self.state)
                    new_state[i][j + 1] = self.state[i][j]
                elif j > 0 and self.state[i][j - 1] == EMPTY:
                    new_state = copy.deepcopy(self.state)
                    new_state[i][j - 1] = self.state[i][j]
                elif i + 1 < len(self.state[j]) and self.state[i + 1][j] == EMPTY:
                    new_state = copy.deepcopy(self.state)
                    new_state[i + 1][j] = self.state[i][j]
                elif i > 0 and self.state[i - 1][j] == EMPTY:
                    new_state = copy.deepcopy(self.state)
                    new_state[i - 1][j] = self.state[i][j]
                if new_state is not None:
                    new_state[i][j] = EMPTY
                    h = self.manhattan_distance(new_state, goal)
                    if heuristic == MANHATTAN_LINEAR_CONFLICT_HEURISTIC:
                        h += self.linear_conflict(new_state, goal)
                    new_states.append(Node(new_state, self.g + 1, h, self))
        return new_states

    def find_index(self, number):
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                if self.state[i][j] == number:
                    return i, j
        return -1, -1

    def manhattan_distance(self, current, other):
        distance = 0
        for i in range(len(current)):
            for j in range(len(current[i])):
                dis_i, dis_j = other.find_index(current[i][j])
                distance += (abs(i - dis_i) + abs(j - dis_j))
        return distance

    def linear_conflict(self, current, other):
        linear_conflict = 0
        for i in range(len(current)):
            for j in range(len(current[i])):
                if current[i][j] == EMPTY:
                    continue
                cell_i, cell_j = other.find_index(current[i][j])
                if (cell_i == i) and (cell_j == j):
                    continue
                if i == cell_i:
                    for col in range(j + 1, len(current)):
                        correct_i, _ = other.find_index(current[i][col])
                        if current[i][j] > current[i][col] and correct_i == i:
                            linear_conflict += 1
                elif j == cell_j:
                    for row in range(i + 1, len(current[i])):
                        _, correct_j = other.find_index(current[row][j])
                        if current[i][j] > current[row][j] and correct_j == j:
                            linear_conflict += 1
        return 2 * linear_conflict

    def find(self, list):
        for i in list:
            if self.state == i.state:
                return i
        return None

    def obtain_path(self, path=None):
        if path is None:
            path = []
        path.append(self)
        if self.parent is None:
            path.reverse()
            return path
        return self.parent.obtain_path(path)


def a_star(start, goal, heuristic):
    queue = [start]
    expanded = []
    time_complexity = 0
    space_complexity = 0
    while queue:
        current = queue.pop()
        if current.manhattan_distance(current.state, goal) == 0:
            print(f'Time complexity = {time_complexity}')
            print(f'Space complexity = {space_complexity}')
            return current.obtain_path()
        if current.find(expanded) is not None:
            continue
        next = current.next_node(goal, heuristic)
        for n in next:
            e = n.find(expanded)
            if (e is not None) and (e.g > n.g):
                expanded.remove(e)
            queue.append(n)
            time_complexity += 1
        expanded.append(current)
        queue.sort(key=lambda x: x.value, reverse=True)
        space_complexity = max(len(queue), len(expanded))


state_1 = [
    [7, 2, 8],
    [4, 5, 6],
    [3, 0, 1]
]

state_2 = [
    [1, 2, 3],
    [7, 6, 5],
    [8, 0, 4]
]

اجرا الگوریتم با استفاده از فاصله منهتن

همانطور که مشاهده میشود، طول مسیر برای رسیدن به حالت هدف برابر با 26 است

In [2]:
path = a_star(Node(state_1, 0, 0, None), Node(state_2, 0, 0, None), MANHATTAN_HEURISTIC)
print(len(path) - 1)
print_path(path)

Time complexity = 5835
Space complexity = 2358
26
7 | 2 | 8
4 | 5 | 6
3 |   | 1
    ↓
7 | 2 | 8
4 | 5 | 6
  | 3 | 1
    ↓
7 | 2 | 8
  | 5 | 6
4 | 3 | 1
    ↓
  | 2 | 8
7 | 5 | 6
4 | 3 | 1
    ↓
2 |   | 8
7 | 5 | 6
4 | 3 | 1
    ↓
2 | 8 |  
7 | 5 | 6
4 | 3 | 1
    ↓
2 | 8 | 6
7 | 5 |  
4 | 3 | 1
    ↓
2 | 8 | 6
7 | 5 | 1
4 | 3 |  
    ↓
2 | 8 | 6
7 | 5 | 1
4 |   | 3
    ↓
2 | 8 | 6
7 |   | 1
4 | 5 | 3
    ↓
2 | 8 | 6
7 | 1 |  
4 | 5 | 3
    ↓
2 | 8 | 6
7 | 1 | 3
4 | 5 |  
    ↓
2 | 8 | 6
7 | 1 | 3
4 |   | 5
    ↓
2 | 8 | 6
7 | 1 | 3
  | 4 | 5
    ↓
2 | 8 | 6
  | 1 | 3
7 | 4 | 5
    ↓
2 | 8 | 6
1 |   | 3
7 | 4 | 5
    ↓
2 |   | 6
1 | 8 | 3
7 | 4 | 5
    ↓
2 | 6 |  
1 | 8 | 3
7 | 4 | 5
    ↓
2 | 6 | 3
1 | 8 |  
7 | 4 | 5
    ↓
2 | 6 | 3
1 | 8 | 5
7 | 4 |  
    ↓
2 | 6 | 3
1 | 8 | 5
7 |   | 4
    ↓
2 | 6 | 3
1 |   | 5
7 | 8 | 4
    ↓
2 |   | 3
1 | 6 | 5
7 | 8 | 4
    ↓
  | 2 | 3
1 | 6 | 5
7 | 8 | 4
    ↓
1 | 2 | 3
  | 6 | 5
7 | 8 | 4
    ↓
1 | 2 | 3
7 | 6 | 5
  | 8 | 4
    ↓
1 | 2 | 3
7 | 

این بار الگوریتم را با جمع فاصله منهتن و دو برابر تعداد تضاد های خطی اجرا میکنیم.

همانطور که در این اجرا مشاهده میشود، استفاده از این تابع باعث میشود مسیر رسیدن به هدف دارای دو گام بیشتر باشد و در 28 قدم به هدف برسیم.

In [3]:
path = a_star(Node(state_1, 0, 0, None), Node(state_2, 0, 0, None), MANHATTAN_LINEAR_CONFLICT_HEURISTIC)
print(len(path) - 1)
print_path(path)

Time complexity = 12087
Space complexity = 4747
28
7 | 2 | 8
4 | 5 | 6
3 |   | 1
    ↓
7 | 2 | 8
4 | 5 | 6
  | 3 | 1
    ↓
7 | 2 | 8
  | 5 | 6
4 | 3 | 1
    ↓
7 | 2 | 8
5 |   | 6
4 | 3 | 1
    ↓
7 | 2 | 8
5 | 6 |  
4 | 3 | 1
    ↓
7 | 2 | 8
5 | 6 | 1
4 | 3 |  
    ↓
7 | 2 | 8
5 | 6 | 1
4 |   | 3
    ↓
7 | 2 | 8
5 | 6 | 1
  | 4 | 3
    ↓
7 | 2 | 8
  | 6 | 1
5 | 4 | 3
    ↓
7 | 2 | 8
6 |   | 1
5 | 4 | 3
    ↓
7 | 2 | 8
6 | 1 |  
5 | 4 | 3
    ↓
7 | 2 |  
6 | 1 | 8
5 | 4 | 3
    ↓
7 |   | 2
6 | 1 | 8
5 | 4 | 3
    ↓
7 | 1 | 2
6 |   | 8
5 | 4 | 3
    ↓
7 | 1 | 2
6 | 8 |  
5 | 4 | 3
    ↓
7 | 1 | 2
6 | 8 | 3
5 | 4 |  
    ↓
7 | 1 | 2
6 | 8 | 3
5 |   | 4
    ↓
7 | 1 | 2
6 |   | 3
5 | 8 | 4
    ↓
7 | 1 | 2
  | 6 | 3
5 | 8 | 4
    ↓
7 | 1 | 2
5 | 6 | 3
  | 8 | 4
    ↓
7 | 1 | 2
5 | 6 | 3
8 |   | 4
    ↓
7 | 1 | 2
5 |   | 3
8 | 6 | 4
    ↓
7 | 1 | 2
  | 5 | 3
8 | 6 | 4
    ↓
  | 1 | 2
7 | 5 | 3
8 | 6 | 4
    ↓
1 |   | 2
7 | 5 | 3
8 | 6 | 4
    ↓
1 | 2 |  
7 | 5 | 3
8 | 6 | 4
    ↓
1 | 2 | 3
7 |

|              Heuristic               | Time Complexity | Space Complexity |
|:------------------------------------:|:---------------:|:----------------:|
|          Manhattan Distance          |      5835       |       2358       |
| Manhattan Distance + Linear Conflict |      12087      |       4747       |

استفاده از تعداد گره های اضافه شونده به Q به طور کلی معیار مناسبی برای اندازه گیری پیچیدگی زمانی نیست.

برای مثال ممکن است از یک تابع ابدایی بسیار مناسبی استفده کنیم که در کمترین تعداد گره اضافه شده به لیست ما را به هدف برساند، ولی پیچیدگی زمانی محاسبه تخمین از مرتبه نمایی باشد. فلذا تعداد گره های اضافه شده به لیست نمیتواند معیار مناسبی باشد.